# Iris Classification (one more time) with Neural Networks

In [3]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

/usr/local/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
np.random.seed(42)

In [5]:
df = pd.read_csv('../Data/iris.csv', header=None)

In [6]:
ds = df.values

In [7]:
X = ds[:,:4].astype(float)
Y = ds[:,4]

### Encoding the classes for a neural net
The output layer is going to be 3 nodes each node corresponding to one of our classes.  So, we'll one-hot the labels so that each column will map to one of the output layers.


In [8]:
encoder = LabelEncoder()
encoder.fit(Y)
enc_Y = encoder.transform(Y)
dummy_y = np_utils.to_categorical(enc_Y)
dummy_y[:10,:]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

### Activation functions

* relu - rectified linear unit 

$$f(x) = \begin{cases}0 &  x < 0 \\ 1 & x >= 1 \end{cases}$$

* softmax - turns the output value of each output layer neuron into a probability that it is 'correct'. In our case the index of the neuron with the highest probability gives us back our original label encoded value for the kind of iris it is.

### Loss

The categorical_crossentropy is a measure of error just like mean squared error that we use with neural networks when we have mulitple classes.


### optimizer

* adam - features an adaptive learning rate

In [9]:
def build_model():
    model = Sequential()
    model.add(Dense(8, input_dim=4, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model


In [10]:
estimator = KerasClassifier(build_fn=build_model, epochs=300, batch_size=5, verbose=0)

In [11]:
kfold = KFold(n_splits=10, shuffle=True)#, random_state=42)

## Evaluate the model using k-Fold cross validation

* Repeat the following K times
  * Take the original data set and break it up into k groups
  * Use k-1 groups as the training set
  * test on on the last group
  * remember which group was used as test this time so a different group can be used next time.

This is even nicer/better than the manual train/test split method as you get to check it on a bunch of different train/test splits.

In [12]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold, n_jobs=4)


In [13]:
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 96.67% (4.47%)


In [17]:
results

array([0.93333334, 1.        , 1.        , 1.        , 1.        ,
       0.93333334, 0.86666667, 1.        , 0.93333334, 1.        ])

## The traditional fit predict route

In [14]:
model = build_model()
model.fit(X, dummy_y,
          batch_size=128, epochs=300,
          verbose=1,
          validation_data=(X, dummy_y))

Train on 150 samples, validate on 150 samples
Epoch 1/300
150/150 [==============================] - 0s 1ms/step - loss: 2.9411 - acc: 0.3333 - val_loss: 2.8514 - val_acc: 0.3333
Epoch 2/300
150/150 [==============================] - 0s 72us/step - loss: 2.8434 - acc: 0.3333 - val_loss: 2.7556 - val_acc: 0.3333
Epoch 3/300
150/150 [==============================] - 0s 74us/step - loss: 2.7482 - acc: 0.3333 - val_loss: 2.6609 - val_acc: 0.3333
Epoch 4/300
150/150 [==============================] - 0s 63us/step - loss: 2.6554 - acc: 0.3333 - val_loss: 2.5691 - val_acc: 0.3333
Epoch 5/300
150/150 [==============================] - 0s 49us/step - loss: 2.5619 - acc: 0.3333 - val_loss: 2.4782 - val_acc: 0.3333
Epoch 6/300
150/150 [==============================] - 0s 48us/step - loss: 2.4719 - acc: 0.3333 - val_loss: 2.3887 - val_acc: 0.3333
Epoch 7/300
150/150 [==============================] - 0s 52us/step - loss: 2.3810 - acc: 0.3333 - val_loss: 2.3009 - val_acc: 0.3333
Epoch 8/300
150/1

150/150 [==============================] - 0s 57us/step - loss: 0.8782 - acc: 0.8267 - val_loss: 0.8728 - val_acc: 0.8400
Epoch 62/300
150/150 [==============================] - 0s 52us/step - loss: 0.8723 - acc: 0.8400 - val_loss: 0.8669 - val_acc: 0.8333
Epoch 63/300
150/150 [==============================] - 0s 52us/step - loss: 0.8665 - acc: 0.8333 - val_loss: 0.8611 - val_acc: 0.8133
Epoch 64/300
150/150 [==============================] - 0s 41us/step - loss: 0.8605 - acc: 0.8133 - val_loss: 0.8552 - val_acc: 0.7800
Epoch 65/300
150/150 [==============================] - 0s 42us/step - loss: 0.8546 - acc: 0.7733 - val_loss: 0.8494 - val_acc: 0.7400
Epoch 66/300
150/150 [==============================] - 0s 45us/step - loss: 0.8495 - acc: 0.7400 - val_loss: 0.8439 - val_acc: 0.7333
Epoch 67/300
150/150 [==============================] - 0s 39us/step - loss: 0.8435 - acc: 0.7333 - val_loss: 0.8385 - val_acc: 0.7067
Epoch 68/300
150/150 [==============================] - 0s 38us/step

150/150 [==============================] - 0s 50us/step - loss: 0.5795 - acc: 0.8200 - val_loss: 0.5769 - val_acc: 0.8400
Epoch 122/300
150/150 [==============================] - 0s 49us/step - loss: 0.5767 - acc: 0.8400 - val_loss: 0.5742 - val_acc: 0.8600
Epoch 123/300
150/150 [==============================] - 0s 44us/step - loss: 0.5740 - acc: 0.8600 - val_loss: 0.5716 - val_acc: 0.8800
Epoch 124/300
150/150 [==============================] - 0s 43us/step - loss: 0.5714 - acc: 0.8800 - val_loss: 0.5691 - val_acc: 0.9000
Epoch 125/300
150/150 [==============================] - 0s 48us/step - loss: 0.5687 - acc: 0.9000 - val_loss: 0.5667 - val_acc: 0.9133
Epoch 126/300
150/150 [==============================] - 0s 44us/step - loss: 0.5664 - acc: 0.9133 - val_loss: 0.5645 - val_acc: 0.9000
Epoch 127/300
150/150 [==============================] - 0s 42us/step - loss: 0.5644 - acc: 0.8933 - val_loss: 0.5625 - val_acc: 0.9200
Epoch 128/300
150/150 [==============================] - 0s 45

150/150 [==============================] - 0s 42us/step - loss: 0.4673 - acc: 0.8800 - val_loss: 0.4660 - val_acc: 0.8800
Epoch 182/300
150/150 [==============================] - 0s 42us/step - loss: 0.4659 - acc: 0.8800 - val_loss: 0.4647 - val_acc: 0.8800
Epoch 183/300
150/150 [==============================] - 0s 41us/step - loss: 0.4646 - acc: 0.8800 - val_loss: 0.4634 - val_acc: 0.8867
Epoch 184/300
150/150 [==============================] - 0s 41us/step - loss: 0.4633 - acc: 0.8867 - val_loss: 0.4621 - val_acc: 0.8867
Epoch 185/300
150/150 [==============================] - 0s 42us/step - loss: 0.4620 - acc: 0.8867 - val_loss: 0.4608 - val_acc: 0.8933
Epoch 186/300
150/150 [==============================] - 0s 39us/step - loss: 0.4607 - acc: 0.8933 - val_loss: 0.4595 - val_acc: 0.8933
Epoch 187/300
150/150 [==============================] - 0s 41us/step - loss: 0.4595 - acc: 0.8933 - val_loss: 0.4583 - val_acc: 0.9000
Epoch 188/300
150/150 [==============================] - 0s 42

150/150 [==============================] - 0s 43us/step - loss: 0.4039 - acc: 0.9533 - val_loss: 0.4031 - val_acc: 0.9533
Epoch 242/300
150/150 [==============================] - 0s 41us/step - loss: 0.4030 - acc: 0.9533 - val_loss: 0.4021 - val_acc: 0.9600
Epoch 243/300
150/150 [==============================] - 0s 42us/step - loss: 0.4020 - acc: 0.9600 - val_loss: 0.4012 - val_acc: 0.9600
Epoch 244/300
150/150 [==============================] - 0s 47us/step - loss: 0.4011 - acc: 0.9600 - val_loss: 0.4002 - val_acc: 0.9600
Epoch 245/300
150/150 [==============================] - 0s 46us/step - loss: 0.4002 - acc: 0.9533 - val_loss: 0.3993 - val_acc: 0.9533
Epoch 246/300
150/150 [==============================] - 0s 48us/step - loss: 0.3992 - acc: 0.9533 - val_loss: 0.3985 - val_acc: 0.9533
Epoch 247/300
150/150 [==============================] - 0s 47us/step - loss: 0.3985 - acc: 0.9467 - val_loss: 0.3977 - val_acc: 0.9200
Epoch 248/300
150/150 [==============================] - 0s 44

In [19]:
score = model.evaluate(X, dummy_y,
                        verbose=0)
score

[0.35371209343274435, 0.9733333333333334]

In [20]:
preds = model.predict_classes(X)

In [26]:
preds[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

predict_proba lets us see the probabilities and how they correspond to the classes

In [25]:
model.predict_proba(X)[:10,:]

array([[9.6882755e-01, 3.0254379e-02, 9.1800821e-04],
       [9.6595043e-01, 3.2416686e-02, 1.6328995e-03],
       [9.5972043e-01, 3.8722940e-02, 1.5566356e-03],
       [9.2314446e-01, 7.3068842e-02, 3.7867420e-03],
       [9.6113563e-01, 3.7802637e-02, 1.0617991e-03],
       [9.4380862e-01, 5.4745786e-02, 1.4455033e-03],
       [9.2853552e-01, 6.8862565e-02, 2.6018852e-03],
       [9.5408779e-01, 4.4298373e-02, 1.6138533e-03],
       [9.2526114e-01, 7.0485041e-02, 4.2537968e-03],
       [9.5523560e-01, 4.2740766e-02, 2.0236825e-03]], dtype=float32)

## References

* [Machine Learning Mastery](https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/)